<a href="https://www.kaggle.com/sanjayanbu/pytorch-mnist?scriptVersionId=88713174" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Importing required libraries

In [1]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision import transforms
from sklearn.model_selection import train_test_split

## Reading the input files

In [2]:
train_data = pd.read_csv("../input/digit-recognizer/train.csv")
test_data = pd.read_csv("../input/digit-recognizer/test.csv")

In [3]:
target = 'label'
num_epochs = 2

In [4]:
train_features = train_data.drop([target], axis=1)
train_labels = train_data[target]

train_features = torch.FloatTensor(train_features.values)
train_labels = torch.LongTensor(train_labels.values)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2)

## Constructing the dataset

In [6]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

In [7]:
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=100)
test_loader = DataLoader(dataset=test_dataset, batch_size=100)

## Building Neural Network

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 15)
        self.fc2 = nn.Linear(15, 23)
        self.fc3 = nn.Linear(23, 10)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = NeuralNetwork()
model.parameters

<bound method Module.parameters of NeuralNetwork(
  (fc1): Linear(in_features=784, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=23, bias=True)
  (fc3): Linear(in_features=23, out_features=10, bias=True)
)>

In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training data

In [11]:
for epoch in range(num_epochs):
    for index, (features, labels) in enumerate(train_loader):
        features = features.reshape(-1, 28 * 28)
        outputs = model(features)
        loss = loss_function(outputs, labels)
        if index % 100 == 1:
            print(f'At epoch {epoch}, loss is {loss}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

At epoch 0, loss is 5.210250377655029
At epoch 0, loss is 1.2706698179244995
At epoch 0, loss is 0.6770404577255249
At epoch 0, loss is 0.8593732714653015
At epoch 1, loss is 0.4765016436576843
At epoch 1, loss is 0.45872634649276733
At epoch 1, loss is 0.38371363282203674
At epoch 1, loss is 0.33038994669914246


## Validation

In [12]:
with torch.no_grad():
    total_samples = 0
    total_correct = 0
    for index, (features, labels) in enumerate(test_loader):
        features = features.reshape(-1, 28 * 28)
        outputs = model(features)
        _, predictions = torch.max(outputs, 1)
        accuracy = (predictions == labels).sum().item()
        total_correct += accuracy
        total_samples += len(labels)
    print(f'Accuracy is {(total_correct / total_samples) * 100}')

Accuracy is 87.67857142857143


In [13]:
validation_features = test_data.drop([target], axis=1, errors='ignore')
validation_tensor = torch.FloatTensor(validation_features.values)

In [14]:
validation_dataset = TensorDataset(validation_tensor)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=100 )

## Predicting the test

In [15]:
img_id_list = []
start_index = 1
pred_list = []
with torch.no_grad():
    for index, features in enumerate(validation_loader):
        features = features[0].reshape(-1, 28 * 28)
        output = model(features)
        _, prediction = torch.max(output, 1)
        end_index = start_index + len(prediction)
        img_id_list.extend(list(range(start_index, end_index)))
        pred_list.extend(list(map(lambda x:x.item(), prediction)))
        start_index += len(prediction)

## Submission

In [16]:
submission_df = pd.DataFrame({'ImageId': img_id_list, 'Label': pred_list})

In [17]:
submission_df.to_csv("submission.csv", index=False)

In [18]:
submission_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,2
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
